## Notebook Lección 2

### Objetivos Notebook

1) Crear funciones desde cero para computar las principales métricas de los portafolios de renta fija.
 
2) Reforzar y practicar lo aprendido en el taller de manejo de datos con Python.

### Ejercicio

Desarrolle los siguientes puntos:

1. Cree funciones para calcular la duración y convexidad para bonos de cupón fijo
2. Cree funciona para calcular el VaR y el CVaR de manera tanto paramétrica como histórica

In [53]:
import sys
import numpy as np
import pandas as pd
import numpy_financial as npf

#### 1. Bonos con Cupón Fijo en Python.

Podemos declarar un bono de cupón fijo como un arreglo de numpy o como una serie de Pandas. En este caso preferiríamos usar series ya que estas cuenta con un índice que nos ayuda a registrar de mejor forma los periodos de pago. Suponga un bono con Face Value de 100 con 6 años de vencimiento que paga cupones anuales:

In [54]:
bono1 = pd.Series([100,100,100,100,100,1100], index = ['1','2','3','4','5','6'], name = 'bono1')
bono1

1     100
2     100
3     100
4     100
5     100
6    1100
Name: bono1, dtype: int64

Supongamos ahora que este bono paga cupónes de forma semianual

In [55]:
bono2 = pd.Series([50,50,50,50,50,50,50,50,50,50,50,1050], 
                  index = ['0.5','1','1.5','2','2.5','3','3.5','4','4.5','5','5.5','6']
                  , name = 'bono2 (semianual)')
bono2

0.5      50
1        50
1.5      50
2        50
2.5      50
3        50
3.5      50
4        50
4.5      50
5        50
5.5      50
6      1050
Name: bono2 (semianual), dtype: int64

Podemos también incluso realizarlos a un vencimiento específico supongamos que este bono fue emitido el 16 de febrero de 2021. 

In [56]:
fechas_cupon = pd.date_range('2021-02-16', periods = 13, freq=pd.DateOffset(months=6), inclusive= 'right')
bono2 = pd.Series([50,50,50,50,50,50,50,50,50,50,50,1050], 
                  index = fechas_cupon
                  , name = 'bono2 (semianual)')
bono2

2021-08-16      50
2022-02-16      50
2022-08-16      50
2023-02-16      50
2023-08-16      50
2024-02-16      50
2024-08-16      50
2025-02-16      50
2025-08-16      50
2026-02-16      50
2026-08-16      50
2027-02-16    1050
Freq: <DateOffset: months=6>, Name: bono2 (semianual), dtype: int64

Este procedimiento lo podemos volver fácilmente una función.

In [57]:
def crearPagos(tasa_cupon, frecuencia, FV, vencimiento):
    indice_bonos = [x*frecuencia for x in range(1,(vencimiento*int(1/frecuencia))+1)]
    serie = pd.Series([tasa_cupon*FV]*(len(indice_bonos)-1)+[(1+tasa_cupon)*FV],indice_bonos)
    return serie
bono = crearPagos(0.05,1/2, 100,6)
bono

0.5      5.0
1.0      5.0
1.5      5.0
2.0      5.0
2.5      5.0
3.0      5.0
3.5      5.0
4.0      5.0
4.5      5.0
5.0      5.0
5.5      5.0
6.0    105.0
dtype: float64

Con respecto al precio del bono podemos tomar el valor de mercado o bien podríamos usar la curva spot para descontar cada cupón a valor presente. Esta opción la revisaremos en la siguiente lección.

#### 2) Precio

El Precio tiene la fórmula que conocemos equivalente a descontar los flujos con la tasa de interés correspondiente: 

$$P = (\frac{F}{(1+y)^T}+\sum_{i=1}^{T}\frac{c\%}{(1+y)^i})$$

También se puede usar la siguiente notación, teniendo en cuenta que gran parte de los bonos tienen cupones de pago semianual:

$$P = \sum_{j=1}^{2T}\frac{c(j/2)}{(1+y/2)^{2j}}$$


#### 3) Yield to Maturity

Conociendo el precio de mercados podemos también calcular el yield to maturity. 

Supongamos un Bono-par a dos (2) años, con pago semestral y cupón del 10% (spot = 10%). 

In [58]:
bono.to_list()

[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 105.0]

Supongamos que el bono se negocia a tasa cupón par respecto a su yield to maturity.

In [59]:
precio = 100
ytm = npf.irr([-precio]+bono.to_list())
round(ytm,2)

0.05

In [60]:
### Este resultado debemos multiplicarlo por dos para obtener la tasa anual semestre vencido
ytm*2

0.10000000000000009

#### 4) Duración

La fórmula general para calcular el duration de un bono de cupón fijo es la siguiente: 

$$D = -\frac{\frac{dV(t,y)}{dy}}{V} = \frac{F}{1+y}(\frac{T}{1+y}+c\%\sum_{i=1}^{T}\frac{i}{(1+y)^i})$$

Particularmente para bonos semimensuales:

$$D = (1+y/2)^{-1}\sum_{i=1}^{2T}\frac{j}{2}\delta(j/2)$$

Donde

$$\delta(j/2) = \frac{(1+y/2)^{-j}c(j/2)}{P}$$

### Ejemplo

Bono par de dos años, con pago semestral y cupón del 10%.

In [61]:
bono = crearPagos(0.05,1/2, 100,2)
bono

0.5      5.0
1.0      5.0
1.5      5.0
2.0    105.0
dtype: float64

Como en este caso el bono es semimensual, es preferible usar la notación semimensual. Debemos entonces cambiar el índice del bono.

In [62]:
bono = bono.rename('Cash Flow').reset_index()
bono['index'] = bono['index']*2
bono = bono.set_index('index', drop = False) # Debemos con
bono

,index,Cash Flow
index,,
1.0,1.0,5.0
2.0,2.0,5.0
3.0,3.0,5.0
4.0,4.0,105.0


Calculamos primero el yield to maturity con la función irr de Numpy-Financial. Note que gracias al uso de la notación semestral, podemos usar la función de irr disponible en Excel o Calculadoras financieras.

In [63]:
P = 100
pagos = [-P]+bono['Cash Flow'].to_list()
yield1 = npf.irr(pagos)
round(yield1,2)

0.05

In [64]:
weights = bono.apply(lambda row: row['Cash Flow'] / (1+yield1)**row['index'], axis=1)/P
weights

index
1.0    0.047619
2.0    0.045351
3.0    0.043192
4.0    0.863838
dtype: float64

In [65]:
duration = (weights*bono['index']*1/2).sum()/(1+yield1)
duration

1.772975252081178

Podemos volver el procedimiento anterior una función

In [66]:
def calcularDuracion(P,bono, frecuencia):
    if(frecuencia == 1/2):
        pagos = [-P]+bono.to_list()
        ### Frecuencia SV 
        yield1 = npf.irr(pagos)    
        bono = bono.rename('Cash Flow').reset_index()
        bono['index'] = bono['index']*2
        weights = bono.apply(lambda row: row['Cash Flow'] / (1+yield1)**row['index'], axis=1)/P
        duration = (weights*bono['index']*1/2).sum()/(1+yield1)
    elif(frecuencia == 1): 
        # Ejercicio #1
        pagos = [-P]+bono.to_list()
    return duration
bono = crearPagos(0.05,1/2, 100,2)
duration = calcularDuracion(100,bono,1/2)
duration

1.772975252081178

In [67]:
crearPagos(0.05,1/2, 100,5)

0.5      5.0
1.0      5.0
1.5      5.0
2.0      5.0
2.5      5.0
3.0      5.0
3.5      5.0
4.0      5.0
4.5      5.0
5.0    105.0
dtype: float64

In [68]:
bono = crearPagos(0.05,1/2, 100,5)
duration = calcularDuracion(100,bono,1/2)
duration

3.8608674645923964

#### 5) Convexidad

En notación semestral: 

$$C = (1+y/2)^{-2}\sum_{j=1}^{2T}\frac{j(j+1)}{4}\delta(j/2)$$

Donde

$$\delta(j/2) = \frac{(1+y/2)^{-j}c(j/2)}{P}$$

In [1]:
def calcularConvexidad(P,bono, frecuencia):
    if(frecuencia == 1/2):
        pagos = [-P]+bono.to_list()
        yield1 = npf.irr(pagos)/(2*frecuencia)
        bono = bono.rename('Cash Flow').reset_index()
        bono['index'] = bono['index']*2
        weights = bono.apply(lambda row: row['Cash Flow'] / (1+yield1)**row['index'], axis=1)/P
        convexity = (weights*bono['index']*(bono['index']+1)*1/4).sum()/((1+yield1)**2)
    elif(frecuencia == 1): 
        # Tarea Adicional
        pagos = [-P]+bono.to_list()

    return convexity

In [70]:
convexidad_bono = calcularConvexidad(100,bono,1/2)
convexidad_bono

18.74942038320432

Le invito a corroborar sus resultados en el siguiente enlace: https://dqydj.com/bond-convexity-calculator/

In [71]:
calcularConvexidad(100,crearPagos(0.05,1/2, 100,2),1/2)

4.118458382885235

### Tarea Adicional

Complete el condicional en la función provista para calcular el convexity y el duration 

Puede usar el siguiente link para comparar sus resultados https://www.buyupside.com/calculators/bondconvexity.html

### 6) VaR y CVaR

Suponga que el retorno del activo A tiene un retorno esperado de 1% y una volatilidad de 1%. Obtenga el VaR y CVaR. Suponga que los retornos del activo A siguen una distribución normal.

In [72]:
from scipy import stats as st
retA = 0.02
volA = 0.02
def calcularVaR(ret,vol):
    VaR = ret + vol*st.norm.ppf(0.05)
    return VaR
VaRA= calcularVaR(retA, volA)

def calcularCVaR(ret, vol):
    CVaR = ret - (vol/0.05)*st.norm.pdf(st.norm.ppf(0.05))
    return CVaR
CVaRA = calcularCVaR(retA, volA)
print(VaRA,CVaRA)

-0.012897072539029459 -0.021254256150148507


Suponga que ahora usted va a ingresar un nuevo activo B al portafolio en una proporción 50 a 50. Este activo tiene un retorno esperado de 12% y una volatilidad del 14%. Suponga que los retornos siguen una distribución normal. Asuma que el coeficiente de correlación es cero.

In [73]:
retB = 0.12
volB = 0.14
corrAB = 0
VaRB = calcularVaR(retB, volB)
CVaRB = calcularCVaR(retB, volB)
print(VaRB, CVaRB)

-0.11027950777320622 -0.16877979305103963


In [74]:
wA = 0.5 
wB = 1-wA
retP = wA*retA+wB*retB
volP = np.sqrt((wA*volA)**2 + (wB*volB)**2+2*wA*wB*corrAB*volA*volB)
VaRP = calcularVaR(retP, volP)
CVaRP = calcularCVaR(retP, volP)
print(VaRP, CVaRP)

-0.04630871536766745 -0.07585582138288426


Asuma ahora que el cociente de correlación es 0.5.

In [75]:
corrAB = 0.5
volP = np.sqrt((wA*volA)**2 + (wB*volB)**2+2*wA*wB*corrAB*volA*volB)
VaRP = calcularVaR(retP, volP)
CVaRP = calcularVaR(retP, volP)
print(VaRP,CVaRP)

-0.05418372553738221 -0.05418372553738221


Asuma ahora que el cociente de correlación es -0.5.

In [76]:
corrAB = -0.5
volP = np.sqrt((wA*volA)**2 + (wB*volB)**2+2*wA*wB*corrAB*volA*volB)
VaRP = calcularVaR(retP, volP)
CVaRP = calcularVaR(retP, volP)
print(VaRP,CVaRP)

-0.037860265402094626 -0.037860265402094626


### Tarea Adicional

Redondeando a 1 decimal (e.g. 10%,20%, 30%, ... 100%). Obtenga la ponderación que maximiza el VaR. Asuma un cociente de correlación de -0.5. La solución está en la siguiente celda


### VaR simulación de Montecarlo

In [77]:
wA = 0.9
retP_opt = wA*retA+wB*retB
volP_opt = np.sqrt((wA*volA)**2 + (wB*volB)**2+2*wA*wB*corrAB*volA*volB)
simulations = np.random.normal(retP_opt, volP_opt, size = 100000)
VaR_MC = np.quantile(simulations, 0.05)

### CVaR simulación de MonteCarlo

In [78]:
CVaR_MC = simulations[simulations <= VaR_MC].sum()/len(simulations[simulations <= VaR_MC])
CVaR_MC

-0.051541137987838014

In [79]:
calcularCVaR(retP_opt, volP_opt)

-0.05186902739481604